In [1]:
!pip install dataset transformers[torch,sentencepiece] trl peft bitsandbytes
!pip install --upgrade tokenizers transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.4/318.4 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.5/322.5 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.2/233.2 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.9/105.9 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.9 MB/s eta 0:00:00
  Attempting uninstall: sqlalchemy
    Fo

In [29]:
from transformers.trainer_utils import set_seed
set_seed(42)

In [30]:
from datasets import load_dataset
dataset = load_dataset("llm-book/oasst1-21k-ja",split="train")
print(dataset)

Dataset({
    features: ['conversation'],
    num_rows: 21164
})


In [4]:
from pprint import pprint
pprint(dataset[0])

{'conversation': [{'content': 'こんにちは！', 'role': 'user'},
                  {'content': 'こんにちは！ご質問やお困りのことがありましたら、何でもご相談ください。何が必要か教えてください。',
                   'role': 'assistant'},
                  {'content': '世界のすべての国をアルファベット順に、それぞれの国の人口を教えてください。',
                   'role': 'user'},
                  {'content': '世界中の国をアルファベット順に並べたリストと、その国の推定人口です：\n'
                              '\n'
                              'アフガニスタン: 38,928,346 アルバニア: 2,877,797 '
                              'アルジェリア：44,344,744 アンドラ: 77,265 アンゴラ: 32,878,272 '
                              'アンティグア・バーブーダ: 97,929 アルゼンチン: 45,195,774 アルメニア: '
                              '2,977,600 オーストラリア: 25,499,884 オーストリア: 9,006,398 '
                              'アゼルバイジャン: 10,134,604 バハマ：393,248 バーレーン: '
                              '1,714,571 バングラデシュ: 164,689,383164,689,383 '
                              'バルバドス: 287,375 ベラルーシ: 9,449,323 ベルギー: '
                              '11,589,623 ベリーズ: 397,628 ベナン: 12,123,200 ブータン: 

In [5]:
# トークナイザー呼び出し
from transformers import AutoTokenizer

base_model_name = "tokyotech-llm/Swallow-7b-hf"
tokenizer = AutoTokenizer.from_pretrained(base_model_name)

tokenizer_config.json:   0%|          | 0.00/773 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/914k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/457 [00:00<?, ?B/s]

In [6]:
# チャットテンプレートを設定
tokenizer.chat_template = """\
{%- for message in messages %}
{%- if message['role'] == 'user' %}
{{ bos_token + 'ユーザ：' + message['content'] + eos_token }}
{%- elif message['role'] == 'assistant' %}
{{ bos_token + 'アシスタント：' + message['content']  + eos_token }}
{%- endif %}
{% if loop.last and add_generation_prompt %}
{{ bos_token + 'アシスタント：' }}
{%- endif %}
{% endfor %}\
"""

In [7]:
# デフォルトではトークナイズかつID化されたトークンのリストが返されるが、
# ここでは、tokenize=Falseとしてトークナイズ前の文字列を返すように設定
chat_text = tokenizer.apply_chat_template(
    dataset[0]["conversation"],tokenize=False
)
# 発話間に改行が含まれないため、見やすくするためにeos_tokenを改行に置換
print(chat_text.replace(tokenizer.eos_token,"\n"))
#print(chat_text)

<s>ユーザ：こんにちは！
<s>アシスタント：こんにちは！ご質問やお困りのことがありましたら、何でもご相談ください。何が必要か教えてください。
<s>ユーザ：世界のすべての国をアルファベット順に、それぞれの国の人口を教えてください。
<s>アシスタント：世界中の国をアルファベット順に並べたリストと、その国の推定人口です：

アフガニスタン: 38,928,346 アルバニア: 2,877,797 アルジェリア：44,344,744 アンドラ: 77,265 アンゴラ: 32,878,272 アンティグア・バーブーダ: 97,929 アルゼンチン: 45,195,774 アルメニア: 2,977,600 オーストラリア: 25,499,884 オーストリア: 9,006,398 アゼルバイジャン: 10,134,604 バハマ：393,248 バーレーン: 1,714,571 バングラデシュ: 164,689,383164,689,383 バルバドス: 287,375 ベラルーシ: 9,449,323 ベルギー: 11,589,623 ベリーズ: 397,628 ベナン: 12,123,200 ブータン: 754,288 ボリビア: 11,673,095 ボスニア・ヘルツェゴビナ: 3,279,000 ボツワナ：2,335,814 ブラジル: 212,559,417 ブルネイ: 441,484 ブルガリア: 7,034,879 ブルキナファソ: 20,903,273 ブルンジ: 11,890,781 カボベルデ: 555,987 カンボジア: 16,205,218 カメルーン: 26,545,863 カナダ：37,742,154 中央アフリカ共和国4,829,767 チャド: 16,425,864 チリ: 19,116,201 中国: 1,439,323,776 コロンビア: 50,882,891 コモロ: 869,601 コンゴ民主共和国：87,534,403 コンゴ共和国5,457,821 コスタリカ5,094,118 コートジボワール: 26,378,274 クロアチア: 4,105,267 キューバ: 11,239,224 キプロス：1,207,359 チェコ：10,708,919 デンマーク：5,792,2025,792,202



In [8]:
# 会話データの末尾のアシスタントの発話を除き、生成を促すための文字列を追加
chat_text = tokenizer.apply_chat_template(
    dataset[0]["conversation"][:-1],
    tokenize=False,
    add_generation_prompt=True,
)
print(chat_text.replace(tokenizer.eos_token,"\n"))

<s>ユーザ：こんにちは！
<s>アシスタント：こんにちは！ご質問やお困りのことがありましたら、何でもご相談ください。何が必要か教えてください。
<s>ユーザ：世界のすべての国をアルファベット順に、それぞれの国の人口を教えてください。
<s>アシスタント：


In [9]:
# チャットテンプレートを適用してトークンIDに変換
tokenized_dataset = [
    tokenizer.apply_chat_template(item["conversation"])
    for item in dataset
]
# トークンIDに変換されたデータセットの先頭を表示
token_ids = tokenized_dataset[0]
print("トークンID：",token_ids)
print("トークン：",tokenizer.convert_ids_to_tokens(token_ids))


トークンID： [1, 39944, 30383, 33328, 30584, 2, 1, 40135, 30383, 33328, 30584, 31622, 32916, 31111, 30697, 36451, 30199, 32002, 30458, 32009, 32006, 32075, 30330, 31502, 39682, 31622, 32277, 32059, 30267, 31502, 30458, 32090, 30412, 32903, 30466, 32059, 30267, 2, 1, 39944, 30383, 32198, 30199, 32668, 30199, 30356, 30396, 39042, 39383, 33045, 30353, 30330, 32728, 30199, 30356, 30199, 35620, 30396, 32903, 30466, 32059, 30267, 2, 1, 40135, 30383, 32198, 30275, 30199, 30356, 30396, 39042, 39383, 33045, 30353, 37231, 30366, 33634, 30364, 30330, 32016, 30356, 30199, 40875, 35620, 32001, 30383, 13, 13, 34874, 41714, 39676, 29901, 29871, 29941, 29947, 29892, 29929, 29906, 29947, 29892, 29941, 29946, 29953, 29871, 33997, 32873, 29901, 29871, 29906, 29892, 29947, 29955, 29955, 29892, 29955, 29929, 29955, 29871, 32513, 32882, 32166, 30383, 29946, 29946, 29892, 29941, 29946, 29946, 29892, 29955, 29946, 29946, 29871, 37329, 30281, 29901, 29871, 29955, 29955, 29892, 29906, 29953, 29945, 29871, 32709, 309

In [10]:
tokenizer.pad_token = tokenizer.unk_token

In [11]:
from trl import DataCollatorForCompletionOnlyLM

bos = tokenizer.bos_token
collator = DataCollatorForCompletionOnlyLM(
    instruction_template=bos + "ユーザ：",
    response_template=bos + "アシスタント：",
    tokenizer=tokenizer,
)
batch = collator(tokenized_dataset[:1])
input_ids = batch["input_ids"][0]
labels = batch["labels"][0]
print("入力トークンID：",input_ids)
print("正解ラベル：",labels)

入力トークンID： tensor([    1, 39944, 30383, 33328, 30584,     2,     1, 40135, 30383, 33328,
        30584, 31622, 32916, 31111, 30697, 36451, 30199, 32002, 30458, 32009,
        32006, 32075, 30330, 31502, 39682, 31622, 32277, 32059, 30267, 31502,
        30458, 32090, 30412, 32903, 30466, 32059, 30267,     2,     1, 39944,
        30383, 32198, 30199, 32668, 30199, 30356, 30396, 39042, 39383, 33045,
        30353, 30330, 32728, 30199, 30356, 30199, 35620, 30396, 32903, 30466,
        32059, 30267,     2,     1, 40135, 30383, 32198, 30275, 30199, 30356,
        30396, 39042, 39383, 33045, 30353, 37231, 30366, 33634, 30364, 30330,
        32016, 30356, 30199, 40875, 35620, 32001, 30383,    13,    13, 34874,
        41714, 39676, 29901, 29871, 29941, 29947, 29892, 29929, 29906, 29947,
        29892, 29941, 29946, 29953, 29871, 33997, 32873, 29901, 29871, 29906,
        29892, 29947, 29955, 29955, 29892, 29955, 29929, 29955, 29871, 32513,
        32882, 32166, 30383, 29946, 29946, 29892, 2994

In [12]:
import itertools

segments_to_fit: list[list[int]] = []
segments_to_ignore: list[list[int]] = []
# ラベルが-100である箇所とそうでない箇所ごとにグルーピング
for key,group in itertools.groupby(
    range(len(input_ids)),key=lambda i: labels[i] == -100
):
    group = list(group)
    if key:
        segments_to_ignore.append(group)
    else:
        segments_to_fit.append(group)

print("----損失を計算しない部分----")
for seg in segments_to_ignore:
    print(tokenizer.decode(input_ids[seg]))
    print()

print("----損失を計算する部分----")
for seg in segments_to_fit:
    print(tokenizer.decode(input_ids[seg]))
    print()




----損失を計算しない部分----
<s>ユーザ：こんにちは！</s><s>アシスタント：

<s>ユーザ：世界のすべての国をアルファベット順に、それぞれの国の人口を教えてください。</s><s>アシスタント：

----損失を計算する部分----
こんにちは！ご質問やお困りのことがありましたら、何でもご相談ください。何が必要か教えてください。</s>

世界中の国をアルファベット順に並べたリストと、その国の推定人口です：

アフガニスタン: 38,928,346 アルバニア: 2,877,797 アルジェリア：44,344,744 アンドラ: 77,265 アンゴラ: 32,878,272 アンティグア・バーブーダ: 97,929 アルゼンチン: 45,195,774 アルメニア: 2,977,600 オーストラリア: 25,499,884 オーストリア: 9,006,398 アゼルバイジャン: 10,134,604 バハマ：393,248 バーレーン: 1,714,571 バングラデシュ: 164,689,383164,689,383 バルバドス: 287,375 ベラルーシ: 9,449,323 ベルギー: 11,589,623 ベリーズ: 397,628 ベナン: 12,123,200 ブータン: 754,288 ボリビア: 11,673,095 ボスニア・ヘルツェゴビナ: 3,279,000 ボツワナ：2,335,814 ブラジル: 212,559,417 ブルネイ: 441,484 ブルガリア: 7,034,879 ブルキナファソ: 20,903,273 ブルンジ: 11,890,781 カボベルデ: 555,987 カンボジア: 16,205,218 カメルーン: 26,545,863 カナダ：37,742,154 中央アフリカ共和国4,829,767 チャド: 16,425,864 チリ: 19,116,201 中国: 1,439,323,776 コロンビア: 50,882,891 コモロ: 869,601 コンゴ民主共和国：87,534,403 コンゴ共和国5,457,821 コスタリカ5,094,118 コートジボワール: 26,378,274 クロアチア: 4,105,267 キューバ: 11,239,224 キプロス：1,207,359 チ

## QLoRAのためのモデルの準備

In [13]:
import torch
from transformers import AutoModelForCausalLM,BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)
model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    torch_dtype=torch.bfloat16,
    quantization_config=quantization_config,
    use_cache=False,
    device_map="auto",
)

config.json:   0%|          | 0.00/685 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/28.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.68G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/203 [00:00<?, ?B/s]

In [15]:
#LoRA
from peft import LoraConfig,TaskType,get_peft_model

#LoRAの設定
peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=128,
    lora_alpha=128,
    lora_dropout=0.05,
    target_modules=["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj",],
)
model.enable_input_require_grads()
model = get_peft_model(model,peft_config)
model.print_trainable_parameters()


trainable params: 319,815,680 || all params: 7,149,785,088 || trainable%: 4.4731


In [16]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [28]:
from transformers import Trainer,TrainingArguments

#学習パラメータ
training_args = TrainingArguments(
    output_dir = "./content/drive/MyDrive/llm_book/IT_results",
    per_device_train_batch_size=2,
    bf16=True,
    gradient_accumulation_steps=8,
    gradient_checkpointing=True,
    num_train_epochs=1.0,
    learning_rate=3e-4,
    optim="paged_adamw_8bit",
    lr_scheduler_type="cosine",
    warmup_ratio=0.1,
    max_grad_norm=0.3,
    logging_steps=10,
    save_steps=300,
)

trainer = Trainer(
    model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=collator,
    tokenizer=tokenizer,
)


In [26]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


Step,Training Loss


OutOfMemoryError: CUDA out of memory. Tried to allocate 1.83 GiB. GPU 0 has a total capacity of 14.75 GiB of which 1.14 GiB is free. Process 12338 has 13.29 GiB memory in use. Of the allocated memory 10.06 GiB is allocated by PyTorch, and 3.10 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)